In [2]:
from pandas_datareader import data
from pandas_datareader._utils import RemoteDataError
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import csv
import re
from datetime import datetime

import os.path
from os import path
import time

# import sklearn
# import scipy

# from sklearn.naive_bayes import GaussianNB
# import lightgbm as lightgbm
# from xgboost import XGBClassifier
# import keras

# Get Started
# https://github.com/Refinitiv-API-Samples/Article.EikonDataAPI.DotNet.Library
# API limits
# https://developers.refinitiv.com/en/api-catalog/eikon/eikon-data-api/documentation?content=49692&type=documentation_item
import eikon as ek 

from pylab import rcParams
plt.rcParams['figure.figsize'] = 16, 8

In [3]:
fontsize_reg = 12
fontsize_title = 16
from pylab import rcParams
rcParams['figure.facecolor'] = '1'
rcParams['figure.figsize'] = [8.0, 3.5]
rcParams['figure.dpi'] = 80
rcParams['savefig.dpi'] = 600

rcParams['font.size'] = 12
rcParams['legend.fontsize'] = 'large'
rcParams['figure.titlesize'] = 'large'

In [94]:
# зададим API ключ
ek.set_app_key('ccebd1ff788d4574818be93e35c4741925ac214e')

In [11]:
rics_german_df = pd.read_excel('C:/DAN/t_systems/trade_project/rics_german_excel.xlsx')
sort_cols = ['DAX', 'German_telecom', 'Company Market Cap']
rics_german_df = rics_german_df.sort_values(by=sort_cols, ascending=False)
rics_german_df.index = range(0, len(rics_german_df))
rics_german_df

,ric,Company Common Name,Country of Exchange,Primary Country of Risk,TRBC Industry Group Name,TRBC Industry Name,Price Close,Company Market Cap,DAX,German_telecom
0,DTEGn.DE,Deutsche Telekom AG,Germany,Germany,Telecommunications Services,Integrated Telecommunications Services,18.032030,8.580246e+10,1,1
1,SAPG.DE,SAP SE,Germany,Germany,Software & IT Services,Software,122.335901,1.501916e+11,1,0
2,SIEGn.DE,Siemens AG,Germany,Germany,Consumer Goods Conglomerates,Consumer Goods Conglomerates,139.347250,1.183675e+11,1,0
3,ALVG.DE,Allianz SE,Germany,Germany,Insurance,Multiline Insurance & Brokers,235.534278,9.704549e+10,1,0
4,VOWG_p.DE,Volkswagen AG,Germany,Germany,Automobiles & Auto Parts,Auto & Truck Manufacturers,182.653284,9.619011e+10,1,0
...,...,...,...,...,...,...,...,...,...,...
1486,IXM0392.DE,MSCI Pac ex Jap ETF,Germany,NaN,NaN,NaN,164.045417,NaN,0,0
1487,IXM0402.DE,MSCI Canada ETF,Germany,NaN,NaN,NaN,147.258410,NaN,0,0
1488,IXM0422.DE,MSCI USA ETF,Germany,NaN,NaN,NaN,368.704250,NaN,0,0
1489,IXM0432.DE,MSCI Jap ETF,Germany,NaN,NaN,NaN,179.058138,NaN,0,0


In [15]:
condition_german = rics_german_df['Country of Exchange'] == 'Germany'
condition_telecom = rics_german_df['TRBC Industry Group Name'] == 'Telecommunications Services'
rics_german_telekom_df = rics_german_df[condition_german & condition_telecom]
rics_german_telekom_df[['ric', 'Company Common Name', 'Company Market Cap', 'DAX', 'German_telecom']]

,ric,Company Common Name,Company Market Cap,DAX,German_telecom
0,DTEGn.DE,Deutsche Telekom AG,8.580246e+10,1,1
27,O2Dn.DE,Telefonica Deutschland Holding AG,8.088962e+09,0,1
28,UTDI.DE,United Internet AG,7.962405e+09,0,1
29,DRIG.DE,1&1 Drillisch AG,4.331541e+09,0,1
30,FNTGn.DE,freenet AG,2.614030e+09,0,1
31,TC1n.DE,Tele Columbus AG,3.688837e+08,0,1
32,NFN.DE,NFON AG,3.245050e+08,0,1
33,E4CG.DE,Ecotel Communication AG,3.558930e+07,0,1
34,PH6.F,Philion SE,2.065664e+05,0,1


In [16]:
condition_dax= rics_german_df['DAX'] == 1
rics_dax = rics_german_df[condition_dax]
rics_dax[['ric', 'Company Common Name', 'Company Market Cap', 'DAX', 'German_telecom']]

,ric,Company Common Name,Company Market Cap,DAX,German_telecom
0,DTEGn.DE,Deutsche Telekom AG,8.580246e+10,1,1
1,SAPG.DE,SAP SE,1.501916e+11,1,0
2,SIEGn.DE,Siemens AG,1.183675e+11,1,0
3,ALVG.DE,Allianz SE,9.704549e+10,1,0
4,VOWG_p.DE,Volkswagen AG,9.619011e+10,1,0
5,DAIGn.DE,Daimler AG,7.532200e+10,1,0
6,BASFn.DE,BASF SE,7.222737e+10,1,0
7,ADSGn.DE,Adidas AG,7.062461e+10,1,0
8,DPWGn.DE,Deutsche Post AG,5.932596e+10,1,0
9,BMWG.DE,Bayerische Motoren Werke AG,5.780708e+10,1,0


In [64]:
def get_name_of_latest_file_in_folder(folder_name):
    files_creation_dates = pd.DataFrame(data = [], columns=[
        "folder_name", "file_name", "num_time", "str_time"
    ])

    files_creation_dates['file_name'] = os.listdir(folder_name)
    files_creation_dates['folder_name'] = folder_name
    files_creation_dates['file_folder_and_name'] = files_creation_dates['folder_name'] + files_creation_dates['file_name']
    files_creation_dates
    for i in range(0, len(files_creation_dates)):
        file_folder_and_name = files_creation_dates['file_folder_and_name'][i]
        files_creation_dates['num_time'][i] = os.path.getctime(file_folder_and_name)
        files_creation_dates['str_time'][i] = time.ctime(os.path.getctime(file_folder_and_name))

    most_latest_file_dir = files_creation_dates.sort_values(by=['num_time'], ascending=False)['file_folder_and_name'].iloc[0]
    
    return most_latest_file_dir

In [118]:
# Создадим df, в который будем сохранять вообще все новости.

# Можем создать пустой df, куда будем добавлять все новости
# all_headlines_df = pd.DataFrame(data = [], columns=["ric", "versionCreated", "headline", "storyId", "sourceCode", "story"])

# Но если мы уже сохраняли файл, достанем самый свежий из папки
most_latest_file = get_name_of_latest_file_in_folder('C:/DAN/t_systems/trade_project/backup_headlines/')
all_headlines_df = pd.read_csv(most_latest_file, sep='~')
print(all_headlines_df.shape)

(3293, 6)


In [119]:
# Создадим подобие logger'а 

# quasi_logger = pd.DataFrame(data = [], columns=[
#     'ric_i', 'ric_now', 'headlines_slice_i', 'headlines_number',
#     'up_to_date_ric', 'story_i', 'story_id_now', 'story_is_success', 
#     'date_time',
# ])

# Так как у нас уже есть созданный logger, подгрузим его
most_latest_file = get_name_of_latest_file_in_folder('C:/DAN/t_systems/trade_project/logger/')
quasi_logger = pd.read_csv(most_latest_file, sep=';')
print(quasi_logger.shape)

(3293, 9)


In [130]:
# Зададим, по какому df мы будет итерировать ric компании (её код на бирже).
rics_to_loop_df = rics_german_telekom_df
rics_to_loop_df = rics_to_loop_df.reset_index(drop=True) # чтобы юзать .loc()

# Зададим переменную, котролирующую количество выгружаемых за раз заголовков
headlines_count_to_request = 100

# Первый цикл будет итерировать компании.
for ric_i in range(0, len(rics_to_loop_df)): # range(0, len(rics_to_loop_df))
    
    # Зададим переменную, которая скажет, какой сейчас итерируется ric.
    ric_now = rics_to_loop_df.loc[ric_i, "ric"]
    print(ric_i, ric_now)
    
    # Цикл, который прогоняет запросы заголовков.
    # Зададим для него параметр. Прибавим +1, если исчерпали новости.
    last_headlines_request = False
    # Будем вести счёт итерацию цикла While для выгрузки заголовков
    headlines_slice_i = 0
    # Для выкачивания заголовков нужно задать дату, до которой делаем запрос.
    # Если такого ric нет в all_headlines_df со всеми новостями,
    # то не будет задавать дату, до которой выкачивать заголовки.
    # Если такой ric есть в all_headlines_df со всеми новостями,
    # то возьмём минимальную дату из all_headlines_df со всеми новостями
    while not last_headlines_request:
        # Проверим, содержит ли all_headlines_df наш текущий ric
        is_this_ric_in_all = ric_now in list(all_headlines_df['ric'])
        print('is_this_ric_in_all:', is_this_ric_in_all, end=' ')
        if is_this_ric_in_all:
            # Если в нашем датасете присутствует этот ric,
            # то продолжаем по нему выгружать, берём мин имеющуюся дату.
            up_to_date_ric = str(all_headlines_df[all_headlines_df['ric'] == ric_now]['versionCreated'].min())
            print('  up_to_date_ric:', up_to_date_ric, end=' ')
        else:
            # None означает, что метод возьмёт текущий момент времени.
            up_to_date_ric = None
            print('  up_to_date_ric:', up_to_date_ric, end=' ')
        
        news_get_df = ek.get_news_headlines(
            query=ric_now,
            count=headlines_count_to_request,
            date_to=up_to_date_ric
        )
        print('news_get_df.shape:', news_get_df.shape[0])
        
        # Подготовим news_get_df для добавления текст новостей story
        # Для этого добавим столбцы ric, story, тзменим название для headline.
        news_get_df = news_get_df.reset_index(drop=True) # чтобы юзать .loc()
        news_get_df['ric'] = ric_now
        news_get_df['story'] = np.nan
        news_get_df['headline'] = news_get_df['text']
        news_get_df = news_get_df.drop(columns=['text'])
        
        # Добавим тексты новостей к news_get_df по storyId
        for story_i in range(0, len(news_get_df)): # range(0, len(news_get_df))
            story_id_now = news_get_df.loc[story_i, 'storyId']
            
            try:
                story_now = ek.get_news_story(story_id_now)
                # Удалим из истории все тильды на всякий случай.
                story_now = story_now.replace("~","!tilda!")
                # Внесём выкаченную историю в df
                news_get_df.loc[story_i, 'story'] = story_now
                # Пометим, что мы успешно выкачали текст новости
                story_is_success = True
                print(story_i, end='; ')
            
            except:
                story_is_success = False
                # Выведем story с ошибкой
                print(
                    'story_is_success:', story_is_success, 
                    '  ric_now:', ric_now,
                    '  ric_i:', ric_i,
                    '  story_i:', story_i,
                    )
                time.sleep(5)
            
            finally:
                # В любом случае внесём в наш квази-логгер наблюдение story.
                quasi_logger = quasi_logger.append({
                    "ric_i": ric_i,
                    "ric_now": ric_now,
                    "headlines_slice_i": headlines_slice_i,
                    "headlines_number": headlines_number,
                    "up_to_date_ric": up_to_date_ric,
                    "story_i": story_i,
                    "story_id_now": story_id_now,
                    "story_is_success": story_is_success,
                    "date_time": str(datetime.today())
                }, ignore_index = True)
        
        # Если загруженных заголовков меньше 100, то мы исчерпали новости
        # и нужно переключиться на ric следующей компании.
        headlines_number = news_get_df.shape[0]
        if headlines_number < 100:
            last_headlines_request = True
        
        headlines_slice_i = headlines_slice_i + 1
        
        # На данный момент news_get_df состоит из заголовков и текстов к ним.
        # Добавим их к большому df со всеми статьями.
        all_headlines_df = all_headlines_df.append(news_get_df, ignore_index = True)
        print(str(datetime.today()), ric_now, 
              ' ric_i:', ric_i,
              ' headline_slice_i:', headlines_slice_i,
              ' headline_number:', headlines_number,
              ' not null:', news_get_df['story'].notna().sum(),
              up_to_date_ric)
    
    # Сохраним версию логгера
    # Подберём такое название, которого нет в dir, чтобы не переписывать файл.
    file_version = 1
    folder_name = 'C:/DAN/t_systems/trade_project/logger/'
    date_now = datetime.today().strftime("%Y-%m-%d")
    file_name = 'quasi_logger' + '_' + str(date_now) + '_v' + str(file_version) + '.csv'
    while file_name in os.listdir(folder_name):
        file_name = 'quasi_logger' + '_' + str(date_now) + '_v' + str(file_version) + '.csv'
        file_version = file_version + 1
    file_path = folder_name + file_name
    quasi_logger.to_csv(file_path, sep = ';', index=False)
    print(file_path)
    
    # Сохраним версию all_headlines_df со всеми новостями
    # Подберём такое название, которого нет в dir, чтобы не переписывать файл.
    file_version = 1
    folder_name = 'C:/DAN/t_systems/trade_project/backup_headlines/'
    date_now = datetime.today().strftime("%Y-%m-%d")
    file_name = 'all_headlines_df' + '_' + str(date_now) + '_v' + str(file_version) + '.csv'
    while file_name in os.listdir(folder_name):
        file_name = 'all_headlines_df' + '_' + str(date_now) + '_v' + str(file_version) + '.csv'
        file_version = file_version + 1
    # C:/DAN/t_systems/trade_project/backup_headlines/all_headlines_df_2020-12-24_v6.csv
    file_path = folder_name + file_name
    all_headlines_df.to_csv(file_path, sep = '~', index=False)
    print(file_path)

0 DTEGn.DE
is_this_ric_in_all: True   up_to_date_ric: 2019-09-24 14:05:48.494000+00:00 

2020-12-28 20:46:23,644 P[13008] [MainThread 12416] Failed to decode response to json: <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd"><html><head><title>Sign In</title><meta http-equiv="X-UA-Compatible" content="IE=EmulateIE7"/><meta http-equiv="content-type" content="text/html; charset=utf-8"/><META page=login><link rel="icon" type="image/vnd.microsoft.icon" href="/favicon.ico"/><link rel="shortcut icon" href="/favicon.ico"/><link rel="stylesheet" type="text/css" href="/auth/css/charcoal_commonstyles_spr.css?v=_3.43.3501.01"/><link rel="stylesheet" type="text/css" href="/auth/css/refinitiv_charcoal_loginstyles_spr.css?v=patchRFCharcoal12-12-20_3.43.3501.01"/><script type="text/javascript" src="/auth/js/signin_spr.js?v=_3.43.3501.01"></script><script type="text/javascript" src="/js/as.js?v=_3.43.3501.01"></script><script src="/auth/js/jquery-1.10.1.min.js"></script><script src="/auth/js/jquery-as.js?v=_3.43.3501.01"></script><script type='text

KeyError: 'headlines'

In [128]:
all_headlines_df.tail(10)

,ric,versionCreated,headline,storyId,sourceCode,story
3287,DTEGn.DE,2019-09-25 08:36:43.294000+00:00,MOODY'S - EMEA TELECOMS PROVIDERS’ OUTLOOK TUR...,urn:newsml:reuters.com:20190925:nMDYbLsZ6T:1,NS:RTRS,"<div class=""storyContent"" lang=""en""><p>For the..."
3288,DTEGn.DE,2019-09-25 05:14:23+00:00,"Guten Morgen von Reuters am Mittwoch, den 25. ...",urn:newsml:reuters.com:20190925:nL5N26F3B7:1,NS:RTRS,"<div class=""storyContent"" lang=""de""><style typ..."
3289,DTEGn.DE,2019-09-24 22:25:00.268000+00:00,Game On (Your Phone): T-Mobile and Microsoft T...,urn:newsml:reuters.com:20190924:nBw9dJwG7a:1,NS:BSW,"<div class=""storyContent"" lang=""en""><style typ..."
3290,DTEGn.DE,2019-09-24 15:40:00+00:00,T-Mobile US Inc at Deutsche Bank Leveraged Fin...,urn:newsml:reuters.com:20190924:nTRN5hCWvY:1,NS:TRANS,"<div class=""storyContent"" lang=""en""><p><a href..."
3291,DTEGn.DE,2019-09-24 15:00:51.614000+00:00,Endress+Hauser entame une collaboration avec D...,urn:newsml:newsroom:20190924:nNRA9s86f3:0,NS:AWP,"<div class=""storyContent"" lang=""fr""><style typ..."
3292,DTEGn.DE,2019-09-24 14:05:48.494000+00:00,(EN) Makedonski Telekom AD - Skopje Annual Gen...,urn:newsml:reuters.com:20190924:nGLF33PTgp:2,NS:GLFILE,"<div class=""storyContent"" lang=""en""><p><a href..."
3293,O2Dn.DE,2020-12-23 21:06:52.240000+00:00,No bottled O2 until further notice: Mexico Cit...,urn:newsml:reuters.com:20201223:nNRAdunypd:1,NS:EFEING,"<div class=""storyContent"" lang=""en""><style typ..."
3294,O2Dn.DE,2020-12-19 10:03:26.224000+00:00,1&1-Streit mit Telefonica um Vorleistungspreis...,urn:newsml:reuters.com:20201219:nNRAdtdfkf:1,NS:DERSPI,"<div class=""storyContent"" lang=""de""><style typ..."
3295,O2Dn.DE,2020-12-19 10:03:26.224000+00:00,1&1-Streit mit Telefonica um Vorleistungspreis...,urn:newsml:reuters.com:20201219:nNRAdtdfkf:1,NS:DERSPI,"<div class=""storyContent"" lang=""de""><style typ..."
3296,O2Dn.DE,2020-12-18 09:35:43+00:00,MÄRKTE-Entscheidung im Preisstreit hilft Drill...,urn:newsml:reuters.com:20201218:nL8N2IY21Y:1,NS:RTRS,"<div class=""storyContent"" lang=""de""><style typ..."


In [123]:
# Сохранить версию all_headlines_df
file_version = 1
folder_name = 'C:/DAN/t_systems/trade_project/backup_headlines/'
date_now = datetime.today().strftime("%Y-%m-%d")
file_name = 'all_headlines_df' + '_' + str(date_now) + '_v' + str(file_version) + '.csv'
while file_name in os.listdir(folder_name):
    file_name = 'all_headlines_df' + '_' + str(date_now) + '_v' + str(file_version) + '.csv'
    file_version = file_version + 1
# C:/DAN/t_systems/trade_project/backup_headlines/all_headlines_df_2020-12-24_v6.csv
file_path = folder_name + file_name
all_headlines_df.to_csv(file_path, sep = '~', index=False)
print(file_path)

C:/DAN/t_systems/trade_project/backup_headlines/all_headlines_df_2020-12-28_v6.csv
